<a href="https://colab.research.google.com/github/JeMinYoo/Aiffel-Quest/blob/master/Chat_bot/Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/transformer_chatbot/data/ChatbotData.csv')

# 데이터 구조 확인
print(df.head())
print(df.info())

                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB
None


In [5]:
# 입력과 출력 데이터를 하나의 텍스트로 결합
df['input_output'] = df['Q'] + " [SEP] " + df['A']

# 토크나이저 로드
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'sep_token': '[SEP]'})

# 텍스트를 토큰화
df['tokens'] = df['input_output'].apply(lambda x: tokenizer.encode(x, truncation=True, max_length=128))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
class ChatbotDataset(Dataset):
    def __init__(self, tokens):
        self.tokens = tokens

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        return torch.tensor(self.tokens[idx], dtype=torch.long)

# Dataset과 DataLoader 생성
dataset = ChatbotDataset(df['tokens'].tolist())
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


In [8]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    batch_padded = pad_sequence(batch, batch_first=True, padding_value=tokenizer.pad_token_id)
    return batch_padded

dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# GPT-2 모델 로드
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

# Fine-tuning 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Fine-tuning 과정
epochs = 3
for epoch in range(epochs):
    for batch in dataloader:
        inputs = batch.to(device)
        outputs = model(inputs, labels=inputs)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 3, Loss: 1.5250484943389893
Epoch 3, Loss: 0.9678086042404175
Epoch 3, Loss: 1.250667929649353
Epoch 3, Loss: 1.1805611848831177
Epoch 3, Loss: 1.2631629705429077
Epoch 3, Loss: 1.182740569114685
Epoch 3, Loss: 0.8312055468559265
Epoch 3, Loss: 1.3656688928604126
Epoch 3, Loss: 1.1226791143417358
Epoch 3, Loss: 1.0696649551391602
Epoch 3, Loss: 1.1618714332580566
Epoch 3, Loss: 1.3877147436141968
Epoch 3, Loss: 1.6475633382797241
Epoch 3, Loss: 1.1553876399993896
Epoch 3, Loss: 1.4155534505844116
Epoch 3, Loss: 1.1942956447601318
Epoch 3, Loss: 1.6487157344818115
Epoch 3, Loss: 0.8642446398735046
Epoch 3, Loss: 1.120359182357788
Epoch 3, Loss: 1.208320140838623
Epoch 3, Loss: 1.0340709686279297
Epoch 3, Loss: 1.1710299253463745
Epoch 3, Loss: 0.8121309876441956
Epoch 3, Loss: 1.395883321762085
Epoch 3, Loss: 1.2415040731430054
Epoch 3, Loss: 1.502548336982727
Epoch 3, Loss: 0.858643651008606
Epoch 3, Loss: 1.4909725189208984
Epoch 3, Loss: 0.95

In [9]:
def generate_response(question):
    input_ids = tokenizer.encode(question + " [SEP]", return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=50, pad_token_id=tokenizer.pad_token_id)
    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

# 예시 응답 생성
user_input = "안녕하세요"
response = generate_response(user_input)
print(f'챗봇: {response}')


챗봇:  안녕하면 좋은 아니까요.


# 성능 개선을 위해 조정

In [10]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
from google.colab import drive

drive.mount('/content/drive')

# 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/transformer_chatbot/data/ChatbotData.csv')

# 데이터 구조 확인
print(df.head())
print(df.info())

# 입력과 출력 데이터를 하나의 텍스트로 결합, [BOS]와 [EOS] 토큰 사용
df['input_output'] = "[BOS] " + df['Q'] + " [SEP] " + df['A'] + " [EOS]"

# 토크나이저 로드
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'sep_token': '[SEP]', 'bos_token': '[BOS]', 'eos_token': '[EOS]'})

# 텍스트를 토큰화
df['tokens'] = df['input_output'].apply(lambda x: tokenizer.encode(x, truncation=True, max_length=128))

class ChatbotDataset(Dataset):
    def __init__(self, tokens):
        self.tokens = tokens

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        return torch.tensor(self.tokens[idx], dtype=torch.long)

# Dataset과 DataLoader 생성
dataset = ChatbotDataset(df['tokens'].tolist())
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=lambda x: pad_sequence(x, batch_first=True, padding_value=tokenizer.pad_token_id))

# GPT-2 모델 로드 (larger version 가능)
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

# Fine-tuning 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

# Optimizer와 Scheduler 설정
optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(dataloader) * 3  # total number of training steps
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Gradient Accumulation 설정
gradient_accumulation_steps = 8
accumulated_loss = 0

# Fine-tuning 과정
epochs = 3
for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
        inputs = batch.to(device)
        outputs = model(inputs, labels=inputs)
        loss = outputs.loss / gradient_accumulation_steps  # gradient accumulation 적용
        loss.backward()

        accumulated_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            print(f'Epoch {epoch + 1}, Step {step + 1}, Loss: {accumulated_loss:.4f}')
            accumulated_loss = 0

def generate_response(question):
    input_ids = tokenizer.encode("[BOS] " + question + " [SEP]", return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=50, pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

# 예시 응답 생성
user_input = "안녕하세요"
response = generate_response(user_input)
print(f'챗봇: {response}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB
None


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Step 8, Loss: 66.9704
Epoch 1, Step 16, Loss: 35.6829
Epoch 1, Step 24, Loss: 15.2728
Epoch 1, Step 32, Loss: 5.5150
Epoch 1, Step 40, Loss: 4.0027
Epoch 1, Step 48, Loss: 3.9034
Epoch 1, Step 56, Loss: 4.0472
Epoch 1, Step 64, Loss: 3.8526
Epoch 1, Step 72, Loss: 3.7754
Epoch 1, Step 80, Loss: 3.7221
Epoch 1, Step 88, Loss: 3.9157
Epoch 1, Step 96, Loss: 3.4209
Epoch 1, Step 104, Loss: 3.5493
Epoch 1, Step 112, Loss: 3.4834
Epoch 1, Step 120, Loss: 3.3856
Epoch 1, Step 128, Loss: 3.4440
Epoch 1, Step 136, Loss: 3.1712
Epoch 1, Step 144, Loss: 2.8867
Epoch 1, Step 152, Loss: 3.0679
Epoch 1, Step 160, Loss: 2.9739
Epoch 1, Step 168, Loss: 2.9393
Epoch 1, Step 176, Loss: 2.7056
Epoch 1, Step 184, Loss: 2.7716
Epoch 1, Step 192, Loss: 2.7217
Epoch 1, Step 200, Loss: 2.8750
Epoch 1, Step 208, Loss: 2.7646
Epoch 1, Step 216, Loss: 2.8096
Epoch 1, Step 224, Loss: 2.6458
Epoch 1, Step 232, Loss: 2.6746
Epoch 1, Step 240, Loss: 2.6716
Epoch 1, Step 248, Loss: 2.6116
Epoch 1, Step 256,

In [17]:
# 예시 입력 문장들
questions = [
    "오늘 날씨는 어때?",
    "너의 이름은 뭐야?",
    "오늘 기분이 어때?",
    "좋아하는 음식이 뭐야?",
    "인공지능이란 무엇인가요?",
    "어떤 영화를 추천해줄 수 있니?",
    "여행을 가고 싶은데 어디가 좋을까?",
]

# 각 질문에 대한 모델의 응답 생성
for question in questions:
    response = generate_response(question)
    print(f'사용자: {question}')
    print(f'챗봇: {response}\n')


사용자: 오늘 날씨는 어때?
챗봇:  오늘 날씨는 것도 좋

사용자: 너의 이름은 뭐야?
챗봇:  이름은 이름이 이름이 아

사용자: 오늘 기분이 어때?
챗봇:  오늘 아니에요. 

사용자: 좋아하는 음식이 뭐야?
챗봇:  좋아하는 음식이 �

사용자: 인공지능이란 무엇인가요?
챗봇:  인상은 인상�

사용자: 어떤 영화를 추천해줄 수 있니?
챗봇:  잘 있을 수 �

사용자: 여행을 가고 싶은데 어디가 좋을까?
챗봇:  사랑�



# 추가 개선

In [18]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from google.colab import drive

drive.mount('/content/drive')

# 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/transformer_chatbot/data/ChatbotData.csv')

# 텍스트 정제 및 전처리 (특수 문자 제거, 공백 정리 등)
df['Q'] = df['Q'].str.replace('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9 ]', '')
df['A'] = df['A'].str.replace('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9 ]', '')

# 입력과 출력 데이터를 하나의 텍스트로 결합, [BOS]와 [EOS] 토큰 사용
df['input_output'] = "[BOS] " + df['Q'] + " [SEP] " + df['A'] + " [EOS]"

# 토크나이저 로드
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')  # GPT-2 Medium 사용
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'sep_token': '[SEP]', 'bos_token': '[BOS]', 'eos_token': '[EOS]'})

# 텍스트를 토큰화, max_length를 조금 늘림
df['tokens'] = df['input_output'].apply(lambda x: tokenizer.encode(x, truncation=True, max_length=256))

class ChatbotDataset(Dataset):
    def __init__(self, tokens):
        self.tokens = tokens

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        return torch.tensor(self.tokens[idx], dtype=torch.long)

# Dataset과 DataLoader 생성
dataset = ChatbotDataset(df['tokens'].tolist())
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=lambda x: pad_sequence(x, batch_first=True, padding_value=tokenizer.pad_token_id))

# GPT-2 모델 로드 (Medium 모델 사용)
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model.resize_token_embeddings(len(tokenizer))

# Fine-tuning 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

# Optimizer와 Scheduler 설정
optimizer = AdamW(model.parameters(), lr=3e-5)
total_steps = len(dataloader) * 5  # total number of training steps, 에포크 수를 늘림
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=total_steps)

# Gradient Accumulation 설정
gradient_accumulation_steps = 8
accumulated_loss = 0

# Fine-tuning 과정
epochs = 5
for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
        inputs = batch.to(device)
        outputs = model(inputs, labels=inputs)
        loss = outputs.loss / gradient_accumulation_steps  # gradient accumulation 적용
        loss.backward()

        accumulated_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            print(f'Epoch {epoch + 1}, Step {step + 1}, Loss: {accumulated_loss:.4f}')
            accumulated_loss = 0

# 응답 생성 함수
def generate_response(question):
    input_ids = tokenizer.encode("[BOS] " + question + " [SEP]", return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Step 8, Loss: 63.1673
Epoch 1, Step 16, Loss: 63.5483
Epoch 1, Step 24, Loss: 60.8855
Epoch 1, Step 32, Loss: 59.6719
Epoch 1, Step 40, Loss: 59.5845
Epoch 1, Step 48, Loss: 60.6335
Epoch 1, Step 56, Loss: 58.3483
Epoch 1, Step 64, Loss: 59.2452
Epoch 1, Step 72, Loss: 58.5662
Epoch 1, Step 80, Loss: 60.6944
Epoch 1, Step 88, Loss: 58.0013
Epoch 1, Step 96, Loss: 59.0666
Epoch 1, Step 104, Loss: 60.5876
Epoch 1, Step 112, Loss: 57.5727
Epoch 1, Step 120, Loss: 57.6918
Epoch 1, Step 128, Loss: 53.3615
Epoch 1, Step 136, Loss: 50.5156
Epoch 1, Step 144, Loss: 51.9908
Epoch 1, Step 152, Loss: 49.2652
Epoch 1, Step 160, Loss: 46.2047
Epoch 1, Step 168, Loss: 49.9504
Epoch 1, Step 176, Loss: 48.3511
Epoch 1, Step 184, Loss: 45.0795
Epoch 1, Step 192, Loss: 44.1185
Epoch 1, Step 200, Loss: 38.3342
Epoch 1, Step 208, Loss: 39.6630
Epoch 1, Step 216, Loss: 38.8785
Epoch 1, Step 224, Loss: 38.5270
Epoch 1, Step 232, Loss: 35.1449
Epoch 1, Step 240, Loss: 32.8472
Epoch 1, Step 248, Loss

In [19]:
# 다양한 입력 문장에 대해 응답 생성
questions = [
    "오늘 날씨는 어때?",
    "너의 이름은 뭐야?",
    "오늘 기분이 어때?",
    "좋아하는 음식이 뭐야?",
    "인공지능이란 무엇인가요?",
    "어떤 영화를 추천해줄 수 있니?",
    "여행을 가고 싶은데 어디가 좋을까?",
]

for question in questions:
    response = generate_response(question)
    print(f'사용자: {question}')
    print(f'챗봇: {response}\n')


사용자: 오늘 날씨는 어때?
챗봇:  저는 거예요. 

사용자: 너의 이름은 뭐야?
챗봇:  이름이 마음이 마음이 있어요. 

사용자: 오늘 기분이 어때?
챗봇:  오늘를 연락이 아니라고 싶어요. 

사용자: 좋아하는 음식이 뭐야?
챗봇:  좋아하는 거예요. 

사용자: 인공지능이란 무엇인가요?
챗봇:  인공지능이란 무엇인가요. 

사용자: 어떤 영화를 추천해줄 수 있니?
챗봇:  그런 안 있는 거예요. 

사용자: 여행을 가고 싶은데 어디가 좋을까?
챗봇:  여행을 가고 싶은데 어디가 좋을까요. 

